In [2]:
from tsfresh.feature_extraction import extract_features
from tsfresh.feature_extraction import settings

import re
import numpy as np
import pandas as pd
import os

In [ ]:
# 压力时序特征处理
count_all = []
fin_alluser = []
files = [x for x in os.listdir('Data/')]
for file in files:
    if re.match(r"^\.",file) is not None:
        files.pop(files.index(file))
user = 0
for file in files:
    pressure_fin1 = []
    pressure_fin2 = []
    pressure_fin3 = []
    fin_all = []
    time_all = []
    time = []
    begintime = 0
    count = 0
    countlines = []
    with open('Data/' + file) as f:
        print(file)
        for line in f:
            if line == '####\n':
                if len(pressure_fin1) > 0:
                    fin1 = np.array(pressure_fin1)
                    fin2 = np.array(pressure_fin2)
                    fin3 = np.array(pressure_fin3)
                    time_once = np.array(time)
                    time_all.append(time_once)
                    countlines.append(count)
                    fin_once = np.column_stack((fin1,fin2,fin3))
#                 print(fin_once.shape, time_once.shape)
#                 fin_once = np.array([fin1,fin2,fin3])
                    fin_all.append(fin_once)
                    pressure_fin1 = []
                    pressure_fin2 = []
                    pressure_fin3 = []
                    count = 0
                    begintime = 0
                    time = []
            else:
                count += 1
                l = re.split(r' ',line)
#             print(l)
                if begintime == 0:
                    begintime = int(l[-1])
                time.append(int(l[-1]) - begintime)
                pressure_fin1.append(float(l[0]))
                pressure_fin2.append(float(l[1]))
                pressure_fin3.append(float(l[2]))         

# 数据截断处理
    sort_count = sorted(countlines)
    count_final = sort_count[4]

    fin_all_final = np.zeros((50,count_final,4)) 


    num = 0
    for array in fin_all:
#     print(array,time_all[0])
        if countlines[num] >= count_final:
            fin_all_final[num] = np.column_stack((array[:count_final,...],time_all[num][:count_final]))
#         print(fin_all_final[num,:5,:])
        else:
            array = np.pad(array,((0,count_final - countlines[num]),(0,0)),'constant',constant_values=(0,0))
            time_all[num] = np.pad(time_all[num],(0,count_final - countlines[num]),'edge')
#         print(array.shape,time_all[num].shape)
            fin_all_final[num] = np.column_stack((array, time_all[num]))
#         print(fin_all_final[num,:-5,:])
        num += 1
    user += 1                                
# print(fin_all_final.shape)
    fin_all_final = np.reshape(fin_all_final,([count_final * 50 , 4]))
# print(fin_all_final.shape)
    fin_all_final = np.column_stack((fin_all_final,np.array([user for x in range(0,count_final*50)])))
    print(fin_all_final.shape)
#     print(count_final*25)
    count_all.append(count_final)
    fin_alluser.append(fin_all_final)
    np.save("NpDf/raw_"+ file.lstrip("Data-").rstrip("txt")+"npy",fin_all_final)
# np.save("./time_zcj.npy",np.array(time))
print("User:" + str(user))
fin_alluser = np.vstack((fin_alluser))
print(fin_alluser.shape)
np.save("fin.npy",fin_alluser)

In [ ]:
# 指尖特征处理(非时序特征，取前几个值取平均)
pos_alluser = []
user = 0
for file in files:
    pos_peruser = []
    pos_fin1x = []
    pos_fin1y = []
    pos_fin2x = []
    pos_fin2y = []
    pos_fin3x = []
    pos_fin3y = []
    count = 0
    with open('Data/'+ file) as f:
        for line in f:
            if line == '####\n':
                if len(pos_fin1x) > 0:
                    x = [0,0,0]
                    y = [0,0,0]
                    num = 0
                    while num < 5:
                        x[0] += pos_fin1x[num]
                        x[1] += pos_fin2x[num]
                        x[2] += pos_fin3x[num]
                        y[0] += pos_fin1y[num]
                        y[1] += pos_fin2y[num]
                        y[2] += pos_fin3y[num]
                        num += 1
                    pos_1x = x[0]/5
                    pos_1y = y[0]/5
                    pos_2x = x[1]/5
                    pos_2y = y[1]/5
                    pos_3x = x[2]/5
                    pos_3y = y[2]/5
                    fin_once = np.array([pos_1x,pos_1y,pos_2x,pos_2y,pos_3x,pos_3y]) # (1,6)
#                 print(fin_once.shape, time_once.shape)
#                 fin_once = np.array([fin1,fin2,fin3])
                    pos_peruser.append(fin_once)
                    pos_fin1x = []
                    pos_fin1y = []
                    pos_fin2x = []
                    pos_fin2y = []
                    pos_fin3x = []
                    pos_fin3y = []
                    count = 0
            else:
                if count < 5 :
                    l = re.split(r' ',line)
                    pos_fin1x.append(float(l[5]))
                    pos_fin1y.append(float(l[6]))
                    pos_fin2x.append(float(l[7]))
                    pos_fin2y.append(float(l[8]))
                    pos_fin3x.append(float(l[9]))
                    pos_fin3y.append(float(l[10]))
                count += 1
    user += 1
    pos_peruser = np.vstack(pos_peruser)
    pos_peruser = np.column_stack((pos_peruser,np.array([user for x in range(0,50)]))) # (50, 7)
#     print(pos_peruser.shape)
    pos_alluser.append(pos_peruser)
    np.save("NpDf/pos_"+ file.lstrip("Data-").rstrip("txt") +"npy",pos_peruser)
pos_alluser = np.vstack(pos_alluser) # (user * 25 ,7)
df_pos = pd.DataFrame(pos_alluser,columns=['pos1x','pos1y','pos2x','pos2y','pos3x','pos3y','id'])
df_pos.to_csv("posall.csv",index=False)
df_pos.shape

In [ ]:
# 压力时序特征提取
# 为每个用户的每个组提取特征
df = pd.DataFrame(fin_alluser, columns=['fin1','fin2','fin3','time','id'])
df.to_csv('findata.csv',index=False)

extract_settings = settings.ComprehensiveFCParameters()

fc_parameters = { "mean": None,
     "standard_deviation": None,
     "mean_abs_change":None,
     "abs_energy":None,
     "rsd":None,
     "autocorrelation":[{'lag': 0}],
     "fft_aggregated":[{"aggtype":'centroid'},
          {"aggtype":'variance'},
          {"aggtype":'skew'},
          {"aggtype":'kurtosis'}],
     "sample_entropy":None                    
    }

df_list = []
group = df.groupby('id')
for count in count_all:
    data = group.get_group(count_all.index(count) + 1)
    id_list = []
    for i in range(0,50):
        for j in range(0,count):
            id_list.append(i)
    data['id'] = id_list
    fea = extract_features(data, column_id='id', column_sort='time',default_fc_parameters=fc_parameters)
    pos = pd.read_csv('posall.csv')
    pos_group = pos.groupby('id')
#     print(pos_group.get_group(count_all.index(count) + 1).head())
#     print(fea.shape)
#     print(pos_group.get_group(count_all.index(count) + 1).shape)
    pos_group = pos_group.get_group(count_all.index(count) + 1)
    pos_group['indexP'] = range(0,50)
    fea['indexF'] = range(0,50)
#     print(pos_group.head())
#     print(fea.head())
    fea = fea.merge(pos_group,left_on='indexF',right_on='indexP')
#     print(fea.head())
    fea.drop(columns=['indexF','indexP'],inplace=True)
    fea = fea.replace([np.inf, -np.inf, 0], np.nan)
    fea = fea.fillna(fea.mean())
#     fea['id'] = [count_all.index(count) + 1 for x in range(0,50)]
    df_list.append(fea)

fea_all = pd.concat(df_list)
fea_all.to_csv('featureall.csv',index=False)

In [18]:
fea_all.dropna(axis=0)

,fin1__mean,fin1__standard_deviation,fin1__mean_abs_change,fin1__abs_energy,fin1__rsd,fin1__autocorrelation__lag_0,"fin1__fft_aggregated__aggtype_""centroid""","fin1__fft_aggregated__aggtype_""variance""","fin1__fft_aggregated__aggtype_""skew""","fin1__fft_aggregated__aggtype_""kurtosis""",...,"fin3__fft_aggregated__aggtype_""skew""","fin3__fft_aggregated__aggtype_""kurtosis""",fin3__sample_entropy,pos1x,pos1y,pos2x,pos2y,pos3x,pos3y,id
0,0.638962,0.137183,0.025136,11.531465,0.214696,1.0,1.721563,9.999471,1.932254,5.986671,...,2.015407,6.301310,0.336472,108.600000,1471.40000,33.000000,1828.00000,82.700000,1019.70000,1.0
1,0.254010,0.012235,0.001211,1.746109,0.048166,1.0,0.363442,2.548256,5.532677,35.715499,...,3.452190,14.580485,1.232144,111.000000,1880.00000,137.845648,1475.20000,98.560016,1017.80000,1.0
2,0.346457,0.021537,0.003028,3.253395,0.062164,1.0,0.519391,3.645868,4.474468,23.747148,...,4.762009,26.405926,0.387116,176.000000,1508.00000,144.964752,1024.07664,142.497584,1857.00000,1.0
3,0.537474,0.039200,0.006663,7.841217,0.072934,1.0,0.699892,4.870886,3.780941,17.602536,...,3.558954,19.614453,0.151107,153.000000,1484.24134,138.000000,1025.00000,103.600000,1833.00000,1.0
4,0.452610,0.123673,0.015445,5.944076,0.273244,1.0,1.361902,8.096698,2.484400,8.574208,...,2.117359,6.134841,0.205444,148.000000,1051.51460,186.000000,1530.00000,95.774674,1841.00000,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45,0.316648,0.011971,0.003028,1.405729,0.037804,1.0,0.163280,0.640176,5.781449,37.041460,...,2.503437,8.339370,1.098612,220.333852,1928.89724,241.537814,1591.00000,99.466148,1079.12738,9.0
46,0.309899,0.071773,0.023622,1.416641,0.231602,1.0,0.818433,2.829663,2.272737,7.227577,...,1.190352,4.268996,0.510826,208.400000,1912.09432,201.200000,1544.40000,86.938343,1067.97742,9.0
47,0.311024,0.139583,0.035130,1.627069,0.448787,1.0,1.098473,3.074892,1.772250,5.557436,...,1.102410,4.444246,0.348307,187.000000,1948.60000,238.703806,1598.73458,84.476775,1055.56900,9.0
48,0.324522,0.093412,0.028468,1.596565,0.287846,1.0,0.835770,2.534635,2.090847,6.461073,...,1.369469,4.846072,0.382992,238.200000,1980.20000,256.505680,1638.20000,104.565332,1069.36532,9.0


In [12]:
fea_all

,fin3__mean,fin3__standard_deviation,fin3__mean_abs_change,fin3__abs_energy,fin3__rsd,fin3__autocorrelation__lag_0,"fin3__fft_aggregated__aggtype_""centroid""","fin3__fft_aggregated__aggtype_""variance""","fin3__fft_aggregated__aggtype_""skew""","fin3__fft_aggregated__aggtype_""kurtosis""",...,"fin2__fft_aggregated__aggtype_""skew""","fin2__fft_aggregated__aggtype_""kurtosis""",fin2__sample_entropy,pos1x,pos1y,pos2x,pos2y,pos3x,pos3y,id
0,0.529600,0.097375,0.021199,7.828879,0.183865,1.0,1.567596,9.114503,2.015407,6.301310,...,3.613774,15.668603,0.171850,108.600000,1471.40000,33.000000,1828.00000,82.700000,1019.70000,1.0
1,0.344415,0.024862,0.005754,3.219480,0.072187,1.0,0.775982,5.548131,3.452190,14.580485,...,12.505584,175.702930,0.396654,111.000000,1880.00000,137.845648,1475.20000,98.560016,1017.80000,1.0
2,0.236220,0.013554,0.003634,1.511563,0.057378,1.0,0.450076,3.121812,4.762009,26.405926,...,3.048894,11.793352,0.579818,176.000000,1508.00000,144.964752,1024.07664,142.497584,1857.00000,1.0
3,0.372703,0.003712,0.000303,3.750883,0.009959,1.0,0.060255,0.435992,3.558954,19.614453,...,2.825236,10.591585,0.510826,153.000000,1484.24134,138.000000,1025.00000,103.600000,1833.00000,1.0
4,0.208516,0.021577,0.006360,1.186496,0.103478,1.0,1.549355,12.062656,2.117359,6.134841,...,2.273816,7.278374,0.105361,148.000000,1051.51460,186.000000,1530.00000,95.774674,1841.00000,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45,0.556243,0.111380,0.026651,4.505363,0.200235,1.0,0.662755,2.198661,2.503437,8.339370,...,3.116786,15.395583,0.646627,220.333852,1928.89724,241.537814,1591.00000,99.466148,1079.12738,9.0
46,0.199663,0.211651,0.046033,1.185256,1.060042,1.0,1.718096,4.420285,1.190352,4.268996,...,1.623354,4.861427,0.492476,208.400000,1912.09432,201.200000,1544.40000,86.938343,1067.97742,9.0
47,0.190101,0.225926,0.050273,1.220534,1.188451,1.0,1.815114,4.386256,1.102410,4.444246,...,1.551978,4.785774,1.098612,187.000000,1948.60000,238.703806,1598.73458,84.476775,1055.56900,9.0
48,0.114736,0.142254,0.025439,0.467605,1.239838,1.0,1.562929,3.676252,1.369469,4.846072,...,1.435641,4.609744,0.847298,238.200000,1980.20000,256.505680,1638.20000,104.565332,1069.36532,9.0
